In [1]:
#won't be running this too due to potato pc problems

In [2]:
import tensorflow as tf
import os
import random
import numpy as np

In [3]:
seed_num = 12020
random.seed(seed_num)
np.random.seed(seed_num)
tf.random.set_seed(seed_num)

In [4]:
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)